Notebook purpose

- Explore additional outcome measures

In [1]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as agg
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [2]:
df = hd.read_txn_data("777")
hd.inspect(df)

Time for read_txn_data                 : 3.13 seconds
(651,789, 18)


,date,user_id,amount,desc,merchant,tag_group,tag,account_id,account_last_refreshed,account_provider,account_type,debit,female,id,latest_balance,postcode,tag_auto,yob
0,2012-02-01,777,400.00,<mdbremoved> - s/o,NaN,transfers,other_transfers,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688300,364.220001,wa1 4,transfers,1969.0
1,2012-02-01,777,3.03,aviva pa - d/d,aviva,spend,finance,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688299,364.220001,wa1 4,health insurance,1969.0


In [8]:
df[df.desc.str.contains("therapy")].desc

116722                                ss massage therapy cd 7345 deb
116762                                ss massage therapy cd 7345 deb
144330    0891 11jun14 , uk physiotherapy , servi , devizes gb - pos
144351    0891 16jun14 , uk physiotherapy , servi , devizes gb - pos
145961    1633 08jun17 , uk physiotherapy , servi , devizes gb - pos
145989    1633 15jun17 , uk physiotherapy , servi , devizes gb - pos
146006                             uk physiotherapy servidevizes gbr
146028                             uk physiotherapy servidevizes gbr
146108    1633 12jul17 , uk physiotherapy , servi , devizes gb - pos
146176    1633 24jul17 , uk physiotherapy , servi , devizes gb - pos
146241                             uk physiotherapy servidevizes gbr
146360    5500 04sep17 , uk physiotherapy , servi , devizes gb - pos
146426    5500 21sep17 , uk physiotherapy , servi , devizes gb - pos
560456                             www.therapy-direct  on 29 aug bcc
560516                            